# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [ ]:
import azureml.core
import pandas as pd
from azureml.core.workspace import Workspace
from azureml.train.automl.run import AutoMLRun
import time
import logging
from sklearn import datasets
from matplotlib import pyplot as plt
from matplotlib.pyplot import imshow
import random
import numpy as np

import os
import pandas as pd
import numpy as np
import json
import requests
import joblib
from sklearn.metrics import confusion_matrix
import itertools
import time
import logging

from azureml.core import Dataset, Workspace, Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.widgets import RunDetails
from azureml.train.automl import AutoMLConfig


from azureml.core.model import Model
from azureml.core.environment import Environment
from azureml.core import Experiment
from azureml.train.estimator import Estimator

# Azure ML Experiment Submission

To use this notebook, you need to download config.json file from Azure ML Workspace and place it in this folder. This will allow us to get the workspace reference right away:

In [ ]:
from azureml.core import Workspace

try:
    ws = Workspace.from_config()
    print(ws.name, ws.location, ws.resource_group, ws.location, sep='\t')
    print('Library configuration succeeded')
except:
    print('Workspace not found')

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# Choose a name for your CPU cluster
cluster_name = "AUTOML-cluster"

# Verify that cluster does not exist already
try:
    cluster = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D12_V2',
                                                           max_nodes=5)
    cluster = ComputeTarget.create(ws, cluster_name, compute_config)

cluster.wait_for_completion(show_output=True)

# Dataset


In [ ]:
#Creating daatset in a file
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
import os
import pickle
import numpy as np

print('fetching MNIST data...')
mnist = fetch_openml('mnist_784')
mnist['target'] = np.array([int(x) for x in mnist['target']])

# use a random subset of n records to reduce training time.
n = 20000
shuffle_index = np.random.permutation(70000)[:n]
X, y = mnist['data'][shuffle_index], mnist['target'][shuffle_index]

os.makedirs('dataset',exist_ok=True)
with open('dataset/mnist.pkl','wb') as f:
    pickle.dump((X,y),f)

print('Done')

In [ ]:
ds = ws.get_default_datastore()
ds.upload('./dataset', target_path='mnist_data', overwrite=True, show_progress=True)

## Training Script

Created a Training Script

In [ ]:
%%writefile mytrain.py
import argparse
import json
import os
from azureml.core import Run
from azureml.core.model import Model
import pickle
import keras
from keras.layers import Dense,Dropout

parser = argparse.ArgumentParser(description='MNIST Train')
parser.add_argument('--data_folder', type=str, dest='data_folder', help='data folder mounting point')
parser.add_argument('--epochs', type=int, default=3)
parser.add_argument('--batch_size', type=int, default=128)
parser.add_argument('--dropout', type=float)
parser.add_argument('--hidden', type=int, default=100)

args = parser.parse_args()

mnist_fn = os.path.join(args.data_folder, 'mnist_data','mnist.pkl')
mnist_fn = 'dataset/mnist.pkl'
with open(mnist_fn,'rb') as f:
    X,y = pickle.load(f)

X /= 255.0
y = keras.utils.to_categorical(y,10)

n = int(0.8*X.shape[0])
x_train = X[0:n]
y_train = y[0:n]
x_test = X[n:]
y_test = y[n:]

model = keras.models.Sequential()
model.add(Dense(args.hidden,input_shape=(784,),activation='relu'))
if args.dropout is not None and args.dropout<1:
    model.add(Dropout(args.dropout))
model.add(Dense(10,activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

model.fit(x_train, y_train,
          batch_size=args.batch_size,
          epochs=args.epochs,
          verbose=1,
          validation_data=(x_test, y_test))

score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

os.makedirs('outputs',exist_ok=True)
model.save('outputs/mnist_model.hdf5')

# Log metrics
run = Run.get_context()
run.log('Test Loss', score[0])
run.log('Accuracy', score[1])

## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [ ]:
experiment_name = 'Keras-MNIST'
exp = Experiment(workspace=ws, name=experiment_name)
script_params = {
                '--data_folder': ws.get_default_datastore(),
}

In [ ]:
estimator = Estimator(source_directory='.',
                script_params=script_params,
                compute_target=cluster,
                entry_script='mytrain.py',
                pip_packages=['keras','tensorflow']
)

In [ ]:
run = exp.submit(estimator)

## Hyperparameter optimization using Hyperdrive


In [ ]:
from azureml.train.hyperdrive import *

param_sampling = RandomParameterSampling({
         '--hidden': choice([50,100,200,300]),
         '--batch_size': choice([64,128]), 
         '--epochs': choice([5,10,50]),
         '--dropout': choice([0.5,0.8,1])
    })

In [ ]:
early_termination_policy = MedianStoppingPolicy(evaluation_interval=1, delay_evaluation=0)
hd_config = HyperDriveConfig(estimator=est,
                            hyperparameter_sampling=param_sampling,
                            policy=early_termination_policy,
                            primary_metric_name='Accuracy',
                            primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                            max_total_runs=16,
                            max_concurrent_runs=4)
experiment = Experiment(workspace=ws, name='keras-hyperdrive')
hyperdrive_run = experiment.submit(hd_config)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [ ]:
# hyperdrive_run.cancel()
from azureml.widgets import RunDetails
RunDetails(hyperdrive_run).show()

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [ ]:
best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
print(best_run)
print('Best accuracy: {}'.format(best_run_metrics['Accuracy']))

In [ ]:
#TODO: Save the best model
best_run.register_model(model_name='mnist_keras', model_path='outputs/mnist_model.hdf5')